# Tutorial 1: Matching

## 0. Import libraries 

In [1]:
from hypex import Matcher

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
from hypex.utils.tutorial_data_creation import create_test_data

In [3]:
df = create_test_data(num_users=10000, rs=42, na_step=45, nan_cols=['age', 'gender'])
df

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
0,0,0,0,504.5,422.777778,NaN,F,Logistics
1,1,4,1,500.0,506.333333,51.0,NaN,E-commerce
2,2,0,0,485.0,434.000000,56.0,F,Logistics
3,3,8,1,452.0,468.111111,46.0,M,E-commerce
4,4,0,0,488.5,420.111111,56.0,M,Logistics
...,...,...,...,...,...,...,...,...
9995,9995,2,1,482.0,501.666667,31.0,M,Logistics
9996,9996,0,0,453.0,406.888889,53.0,M,Logistics
9997,9997,0,0,461.0,415.111111,52.0,F,E-commerce
9998,9998,10,1,491.5,439.222222,22.0,M,E-commerce


In [4]:
df.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'gender', 'industry'],
      dtype='object')

In [5]:
df['treat'].value_counts()

treat
0    5002
1    4998
Name: count, dtype: int64

In [6]:
df['gender'].isna().sum()

223

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [7]:
info_col = ['user_id']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [8]:
# Standard model with base parameters
model = Matcher(input_data=df, outcome=outcome, treatment=treatment, info_col=info_col)

[05.03.2024 16:38:18 | hypex | INFO]: Number of NaN values filled with zeros: 446


**Feature selection** models the significance of features for the accuracy of target approximation. However, it does not
rule out the possibility of overlooked features, the complex impact of features on target description, or the
significance of features from a business logic perspective. The algorithm will not function correctly if there are data
leaks. 
Points to consider when selecting features:

* Data leaks - these should not be present.
* Influence on treatment distribution - features should not affect the treatment distribution.
* The target should be describable by features.
* All features significantly affecting the target should be included.
* The business rationale of features.
* The feature selection function can be useful for addressing these tasks, but it does not solve them nor does it
  absolve the user of the responsibility for their selection, nor does it justify it.

[Link to ReadTheDocs](https://hypex.readthedocs.io/en/latest/pages/modules/selectors.html#selector-classes)

In [9]:
selected_features = model.feature_select()
selected_features

C:\Users\User\PycharmProjects\HypEx1\hypex\selectors\feature_selector.py:42: UserWarning: FeatureSelector does not rule out the possibility of overlooked features, the complex impact of features on target description, or the significance of features from a business logic perspective.
  warnings.warn(


,rank
signup_month,1
pre_spends,2
age,3
gender_F,4
gender_M,5
industry_Logistics,6


In [10]:
chosen_features = selected_features[:4].index
chosen_features

Index(['signup_month', 'pre_spends', 'age', 'gender_F'], dtype='object')

In [11]:
results, quality_results, df_matched = model.estimate(features=chosen_features)

Get treated index: 100%|██████████| 10000/10000 [00:00<00:00, 15390.91it/s] 
[05.03.2024 16:38:30 | Faiss hypex | INFO]: The entry of bias into the ATT is 0.1%


In [12]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,82.613694,2.338178,0.0,78.030866,87.196523,post_spends
ATC,101.657691,4.569680,0.0,92.701118,110.614264,post_spends
ATT,63.554457,0.691298,0.0,62.199513,64.909401,post_spends


In [13]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [14]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.418144e-01,2.209790e-07
pre_spends,2.348715e-264,3.829212e-19
signup_month,0.000000e+00,0.000000e+00


In [15]:
df_matched

,index,signup_month,pre_spends,age,gender_F,gender_M,industry_Logistics,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_Logistics_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,1,4,500.0,51.0,0,0,0,0.0,509.5,51.0,0.0,1.0,0.0,[764],506.333333,421.555556,84.816667,1,0
1,3,8,452.0,46.0,0,1,0,0.0,471.0,47.0,0.0,1.0,1.0,[8031],468.111111,411.777778,56.423533,1,0
2,9,11,476.5,34.0,0,1,0,0.0,504.0,34.0,0.0,1.0,0.0,[4073],441.888889,426.666667,15.334797,1,0
3,11,8,498.5,60.0,1,0,0,0.0,517.0,61.0,1.0,0.0,0.0,[2371],461.444444,413.111111,48.421487,1,0
4,14,8,497.0,24.0,0,1,1,0.0,515.0,24.0,0.0,1.0,0.0,[1920],461.777778,419.000000,42.851463,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,9992,0,456.5,35.0,1,0,1,2.0,451.0,39.0,1.0,0.0,0.0,[7366],404.888889,525.111111,122.146611,0,1
4998,9993,0,503.0,21.0,1,0,1,1.0,502.5,21.0,1.0,0.0,1.0,[6634],431.888889,533.444444,102.555934,0,1
4999,9994,0,480.0,51.0,0,1,0,2.0,476.0,49.0,0.0,1.0,0.5,"[6831, 7432]",411.444444,517.444444,107.788727,0,1
5000,9996,0,453.0,53.0,0,1,1,2.0,447.0,58.0,0.0,1.0,1.0,[6711],406.888889,519.444444,114.490640,0,1


In [16]:
df_matched[df_matched['industry_Logistics'] != df_matched['industry_Logistics_matched']]

,index,signup_month,pre_spends,age,gender_F,gender_M,industry_Logistics,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_Logistics_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
1,3,8,452.0,46.0,0,1,0,0.0,471.0,47.0,0.0,1.0,1.0,[8031],468.111111,411.777778,56.423533,1,0
4,14,8,497.0,24.0,0,1,1,0.0,515.0,24.0,0.0,1.0,0.0,[1920],461.777778,419.000000,42.851463,1,0
6,22,2,482.0,50.0,0,1,0,0.0,487.0,50.0,0.0,1.0,1.0,[9205],519.777778,415.111111,104.687135,1,0
10,30,11,504.5,38.0,0,1,1,0.0,530.0,39.0,0.0,1.0,0.0,[4514],433.222222,419.333333,14.005698,1,0
21,52,4,487.0,38.0,1,0,1,0.0,497.0,38.0,1.0,0.0,0.5,"[452, 7221]",522.444444,432.444444,90.040936,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,9981,0,458.0,36.0,1,0,1,2.0,451.0,39.0,1.0,0.0,0.0,[7366],422.666667,525.111111,104.262818,0,1
4996,9986,0,486.5,53.0,0,1,1,2.0,481.5,53.0,0.0,1.0,0.0,[4106],418.111111,517.888889,101.587895,0,1
4997,9992,0,456.5,35.0,1,0,1,2.0,451.0,39.0,1.0,0.0,0.0,[7366],404.888889,525.111111,122.146611,0,1
4999,9994,0,480.0,51.0,0,1,0,2.0,476.0,49.0,0.0,1.0,0.5,"[6831, 7432]",411.444444,517.444444,107.788727,0,1


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [17]:
group_col = "industry"

Also group_col might be the list. 

In [18]:
model = Matcher(input_data=df, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

[05.03.2024 16:38:31 | hypex | INFO]: Number of NaN values filled with zeros: 446


In [19]:
selected_features = model.feature_select()
selected_features

C:\Users\User\PycharmProjects\HypEx1\hypex\selectors\feature_selector.py:42: UserWarning: FeatureSelector does not rule out the possibility of overlooked features, the complex impact of features on target description, or the significance of features from a business logic perspective.
  warnings.warn(


,rank
signup_month,1
pre_spends,2
age,3
gender_F,4
gender_M,5


In [20]:
chosen_features = selected_features[:4].index
chosen_features

Index(['signup_month', 'pre_spends', 'age', 'gender_F'], dtype='object')

In [21]:
results, quality_results, df_matched = model.estimate(features=chosen_features)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\User\PycharmProjects\HypEx1\hypex\algorithms\faiss_matcher.py:627: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_c = np.cov(xc, rowvar=False, ddof=0)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\User\PycharmProjects\HypEx1\hypex\algorithms\faiss_matcher.py:628: RuntimeWarning: Degrees

In [22]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,81.186361,1.626949,0.0,77.997541,84.375181,post_spends
ATC,98.880907,3.108746,0.0,92.787765,104.974050,post_spends
ATT,63.477653,0.650095,0.0,62.203466,64.751840,post_spends


In [23]:
df_matched[df_matched['industry'] != df_matched['industry_matched']]

,index,signup_month,pre_spends,age,gender_F,gender_M,industry,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched


## 3. Results  
### 3.1 ATE, ATT, ATC

In [24]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,81.186361,1.626949,0.0,77.997541,84.375181,post_spends
ATC,98.880907,3.108746,0.0,92.787765,104.974050,post_spends
ATT,63.477653,0.650095,0.0,62.203466,64.751840,post_spends


### 3.2 SMD, PSI, KS-test, repeats

In [25]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [26]:
quality_results['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.01,OK,age_untreated,0.08,OK
1,gender_F_treated,0.00,OK,gender_F_untreated,0.00,OK
2,industry_treated,0.00,OK,industry_untreated,0.00,OK
3,pre_spends_treated,0.61,NOK,pre_spends_untreated,0.16,OK
4,signup_month_treated,16.14,NOK,signup_month_untreated,0.00,OK


In [27]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.292517e-01,2.721265e-04
pre_spends,4.164710e-263,1.725086e-24
signup_month,0.000000e+00,0.000000e+00


In [28]:
quality_results['repeats']

{'match_control_to_treat': 0.44, 'match_treat_to_control': 0.07}

### 3.3 Validation
Validates estimated effect:
1. by replacing real treatment (`random_treatment`) with random placebo treatment.
 Estimated effect must be droped to zero;
2. by adding random feature (`random_feature`). Estimated effect shouldn't change
significantly, p-val < 0.05;
3. estimates effect on subset of data (`subset_refuter`) (default fraction is 0.8). Estimated effect
shouldn't change significantly, p-val < 0.05.


**Random Treatment** algorithm randomly shuffles the actual treatment. It is expected that the treatment's effect on the target will be close to 0.

**Random Feature** adds a feature with random values. It is expected that adding a random feature will maintain the same impact of the treatment on the target.

These methods are not sufficiently accurate markers of a successful experiment.

[Link to ReadTheDocs](https://hypex.readthedocs.io/en/latest/pages/modules/utils.html#validators)

In [29]:
model.validate_result(refuter="random_treatment", effect_type="att", n_sim=10)

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\User\PycharmProjects\HypEx1\hypex\algorithms\faiss_matcher.py:627: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov_c = np.cov(xc, rowvar=False, ddof=0)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\User\PycharmProjects\HypEx1\hypex\algorithms\faiss_m

{'post_spends': [-0.008625154338610733, 0.0]}

## 4. Save model

In [30]:
model.save("test_model.pickle")

In [31]:
model2 = Matcher.load("test_model.pickle")

In [32]:
model2.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,81.186361,1.626949,0.0,77.997541,84.375181,post_spends
ATC,98.880907,3.108746,0.0,92.787765,104.974050,post_spends
ATT,63.477653,0.650095,0.0,62.203466,64.751840,post_spends


In [33]:
model.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,81.186361,1.626949,0.0,77.997541,84.375181,post_spends
ATC,98.880907,3.108746,0.0,92.787765,104.974050,post_spends
ATT,63.477653,0.650095,0.0,62.203466,64.751840,post_spends
